In [291]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn import model_selection
import sklearn.linear_model as lm

# **Data**

In [294]:
data = pd.read_csv("HR_data.csv")
data = data.drop("Unnamed: 0", axis=1)
data = data.drop("Individual", axis=1)
data = data.drop("Puzzler", axis=1)
data = data.drop("Round", axis=1)
data = data.drop("Phase", axis=1)
data = data.drop("Cohort", axis=1)
X = data.drop("Frustrated", axis=1).to_numpy().astype(np.float32)
Y = data["Frustrated"].to_numpy()
y_val = data["Frustrated"].to_numpy()
num_classes = 11
Y = np.eye(num_classes)[Y]
y_val_tensor = torch.tensor(y_val)
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(Y, dtype=torch.float32)

# **ANN**

In [190]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt

# H is hidden dimension
H = 5
K = 14 # folds

# Device to use for computations
device = torch.device('cpu')

test_pred_ANN = []
CV = model_selection.KFold(n_splits=K, shuffle=False)
for train_index, test_index in CV.split(X_tensor):

    X_tensor_train = X_tensor[train_index]
    y_tensor_train = y_val_tensor[train_index]

    X_tensor_test = X_tensor[test_index]
    y_tensor_test = y_val_tensor[test_index]

    # Normalize X
    train_mean = X_tensor_train.mean(dim=0)
    train_std = X_tensor_train.std(dim=0)
    train_std = torch.where(train_std == 0, torch.tensor(1.0, device=device), train_std)
    X_tensor_train = (X_tensor_train - train_mean) / train_std
    X_tensor_test = (X_tensor_test - train_mean) / train_std

    # Define model
    model = nn.Sequential(
        nn.Linear(6, H),
        nn.ReLU(),
        nn.Linear(H, H),
        nn.ReLU(),
        nn.Linear(H, 11),
        nn.Dropout(0.3)
    )
    model.to(device)
    loss_fn = nn.CrossEntropyLoss()

    # Optimizer
    learning_rate = 1e-4
    optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    # DataLoader for training
    batch_size = 32
    dataset = TensorDataset(X_tensor_train, y_tensor_train)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Number of iterations and early stopping
    T = 1500
    Loss_train = np.zeros(T)
    Loss_test = np.zeros(T)
    Acc_train = []
    Acc_test = []
    best_test_loss = float('inf')
    patience = 50
    trigger_times = 0

    # Training loop
    for t in range(T):
        epoch_loss_train = 0.0
        model.train()
        for X_batch, y_batch in dataloader:
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch)  # y_batch is now class indices
            epoch_loss_train += loss.item() * X_batch.size(0)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        Loss_train[t] = epoch_loss_train / len(dataset)
        
        # Compute test loss
        model.eval()
        with torch.no_grad():
            y_pred_test = model(X_tensor_test)
            loss_test = loss_fn(y_pred_test, y_tensor_test)
            Loss_test[t] = loss_test.item() if not torch.isnan(loss_test).any() else np.nan
        
        if (t + 1) % 500 == 0:
            print(f'Fold: {test_index[0]//12 + 1}, Epoch {t+1}, Train Loss: {Loss_train[t]:.4f}, Test Loss: {Loss_test[t]:.4f}')

    # Predictions:
    for val in range(len(X_tensor_test)):
        test_pred_ANN.append(np.argmax(model(X_tensor_test).tolist()[val]))




Fold: 1, Epoch 500, Train Loss: 2.0829, Test Loss: 2.0018
Fold: 1, Epoch 1000, Train Loss: 2.0319, Test Loss: 1.9422
Fold: 1, Epoch 1500, Train Loss: 1.9606, Test Loss: 1.9246
Fold: 2, Epoch 500, Train Loss: 2.1218, Test Loss: 2.1711
Fold: 2, Epoch 1000, Train Loss: 2.0538, Test Loss: 1.9644
Fold: 2, Epoch 1500, Train Loss: 2.0047, Test Loss: 1.9165
Fold: 3, Epoch 500, Train Loss: 2.0606, Test Loss: 2.0985
Fold: 3, Epoch 1000, Train Loss: 2.0182, Test Loss: 2.0033
Fold: 3, Epoch 1500, Train Loss: 2.0214, Test Loss: 1.9677
Fold: 4, Epoch 500, Train Loss: 2.0411, Test Loss: 2.2279
Fold: 4, Epoch 1000, Train Loss: 1.9506, Test Loss: 2.1618
Fold: 4, Epoch 1500, Train Loss: 1.9674, Test Loss: 2.1559
Fold: 5, Epoch 500, Train Loss: 2.0538, Test Loss: 2.0449
Fold: 5, Epoch 1000, Train Loss: 2.0018, Test Loss: 1.9618
Fold: 5, Epoch 1500, Train Loss: 1.9668, Test Loss: 1.9371
Fold: 6, Epoch 500, Train Loss: 1.9836, Test Loss: 2.3189
Fold: 6, Epoch 1000, Train Loss: 2.0130, Test Loss: 2.2960
Fol

In [298]:
# Predictions from trained ANN (42 correct)
test_pred_ANN = [1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 2, 0, 1, 1, 0, 0, 1, 0, 2, 1, 1, 1, 2, 2, 1, 2, 2, 2, 2, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1]

In [301]:
counter_ANN = 0
for i in range(len(test_pred_ANN)):
    counter_ANN += (y_val[i] == test_pred_ANN[i])
print("---------------TEST---------------")
print("Correct", counter_ANN)
print("Accuracy", counter_ANN/len(X_tensor))

---------------TEST---------------
Correct 42
Accuracy 0.25


## **Logistic regression**

In [302]:
import numpy as np
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import torch

K = 14  # folds
device = torch.device('cpu')
test_pred_lr = []

CV = model_selection.KFold(n_splits=K, shuffle=False)

# # Verify class indices
# assert y_val_tensor.min() >= 0 and y_val_tensor.max() <= 10, "Invalid class indices"

for train_index, test_index in CV.split(X_tensor):
    # Split data
    X_tensor_train = X_tensor[train_index]
    y_tensor_train = y_val_tensor[train_index]
    X_tensor_test = X_tensor[test_index]

    # Normalize X
    train_mean = X_tensor_train.mean(dim=0)
    train_std = X_tensor_train.std(dim=0)
    train_std = torch.where(train_std == 0, torch.tensor(1.0, device=device), train_std)
    X_tensor_train = (X_tensor_train - train_mean) / train_std
    X_tensor_test = (X_tensor_test - train_mean) / train_std

    X_tensor_train_np = X_tensor_train.cpu().numpy()
    y_tensor_train_np = y_tensor_train.cpu().numpy()
    X_tensor_test_np = X_tensor_test.cpu().numpy()

    # Train model
    model = LogisticRegression(max_iter=1000) 
    model.fit(X_tensor_train_np, y_tensor_train_np)

    # Predictions
    predictions = model.predict(X_tensor_test_np)
    test_pred_lr.extend(predictions.tolist())


In [303]:
counter_lr = 0
for i in range(len(test_pred_lr)):
    counter_lr += (y_val[i] == test_pred_lr[i])
print("---------------TEST---------------")
print("Correct", counter_lr)
print("Accuracy", counter_lr/len(X_tensor))

---------------TEST---------------
Correct 29
Accuracy 0.17261904761904762


In [250]:
baseline = np.ones(168)
counter_baseline = 45

# **Test**

### **Confidence intervals (One sample)**

In [255]:
from statsmodels.stats.proportion import proportion_confint
alpha = 0.05
n = len(X)

# ANN
ci_lower, ci_upper = proportion_confint(counter_ANN ,n,method="wilson", alpha=alpha)
print(f"95% CI for accuracy of ANN: [{ci_lower:.4f}, {ci_upper:.4f}]")

# LR
ci_lower, ci_upper = proportion_confint(counter_lr ,n,method="wilson", alpha=alpha)
print(f"95% CI for accuracy of LR: [{ci_lower:.4f}, {ci_upper:.4f}]")

# Baseline
ci_lower, ci_upper = proportion_confint(counter_baseline ,n,method="wilson", alpha=alpha)
print(f"95% CI for accuracy of Baseline: [{ci_lower:.4f}, {ci_upper:.4f}]")

95% CI for accuracy of ANN: [0.1906, 0.3206]
95% CI for accuracy of LR: [0.1230, 0.2369]
95% CI for accuracy of Baseline: [0.2066, 0.3395]


### **McNemar**

#### **ANN vs. LR**

In [305]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np
from scipy.stats import norm

# Compute the contingency table
table = np.array([
    [np.sum((y_val == test_pred_ANN) & (y_val == test_pred_lr)),  # Both correct
     np.sum((y_val == test_pred_ANN) & (y_val != test_pred_lr))],  # ANN correct, LR wrong
    [np.sum((y_val != test_pred_ANN) & (y_val == test_pred_lr)),  # ANN wrong, LR correct
     np.sum((y_val != test_pred_ANN) & (y_val != test_pred_lr))]   # Both wrong
])

# Print the contingency table for debugging
print("Contingency Table:")
print(table)

# Extract discordant pairs
b = table[0, 1]  # ANN correct, LR wrong
c = table[1, 0]  # ANN wrong, LR correct
n = len(y_val)   # Total number of samples

# Perform McNemar's test
result = mcnemar(table, exact=True)
print(f'McNemar p-value (ANN vs LR): {result.pvalue:.4f}')

# Calculate CI for the difference in proportions with continuity correction
diff = (b - c) / n  # Difference in proportions
z = norm.ppf(0.975)  # 97.5th percentile for 95% CI (two-tailed)
ci_lower = diff - z*np.sqrt((b+c-(b-c)**2/n)/(n**2))
ci_upper = diff + z*np.sqrt((b+c-(b-c)**2/n)/(n**2))

print(f'95% CI for difference in proportions (ANN - LR): [{ci_lower:.4f}, {ci_upper:.4f}]')

Contingency Table:
[[ 21  21]
 [  8 118]]
McNemar p-value (ANN vs LR): 0.0241
95% CI for difference in proportions (ANN - LR): [0.0157, 0.1391]


#### **ANN vs. Baseline**

In [306]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np
from scipy.stats import norm

# Compute the contingency table
table = np.array([
    [np.sum((y_val == test_pred_ANN) & (y_val == baseline)),  # Both correct
     np.sum((y_val == test_pred_ANN) & (y_val != baseline))],  # ANN correct, LR wrong
    [np.sum((y_val != test_pred_ANN) & (y_val == baseline)),  # ANN wrong, LR correct
     np.sum((y_val != test_pred_ANN) & (y_val != baseline))]   # Both wrong
])

# Print the contingency table for debugging
print("Contingency Table:")
print(table)

# Extract discordant pairs
b = table[0, 1]  # ANN correct, LR wrong
c = table[1, 0]  # ANN wrong, LR correct
n = len(y_val)   # Total number of samples

# Perform McNemar's test
result = mcnemar(table, exact=True)
print(f'McNemar p-value (ANN vs baseline): {result.pvalue:.4f}')

# Calculate CI for the difference in proportions with continuity correction
diff = (b - c) / n  # Difference in proportions
z = norm.ppf(0.975)  # 97.5th percentile for 95% CI (two-tailed)
ci_lower = diff - z*np.sqrt((b+c-(b-c)**2/n)/(n**2))
ci_upper = diff + z*np.sqrt((b+c-(b-c)**2/n)/(n**2))

print(f'95% CI for difference in proportions (ANN - Baseline): [{ci_lower:.4f}, {ci_upper:.4f}]')

Contingency Table:
[[ 32  10]
 [ 13 113]]
McNemar p-value (ANN vs baseline): 0.6776
95% CI for difference in proportions (ANN - Baseline): [-0.0737, 0.0380]


#### **LR vs. Baseline**

In [307]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np
from scipy.stats import norm

# Compute the contingency table
table = np.array([
    [np.sum((y_val == test_pred_lr) & (y_val == baseline)),  # Both correct
     np.sum((y_val == test_pred_lr) & (y_val != baseline))],  # ANN correct, LR wrong
    [np.sum((y_val != test_pred_lr) & (y_val == baseline)),  # ANN wrong, LR correct
     np.sum((y_val != test_pred_lr) & (y_val != baseline))]   # Both wrong
])

# Print the contingency table for debugging
print("Contingency Table:")
print(table)

# Extract discordant pairs
b = table[0, 1]  # ANN correct, LR wrong
c = table[1, 0]  # ANN wrong, LR correct
n = len(y_val)   # Total number of samples

# Perform McNemar's test
result = mcnemar(table, exact=True)
print(f'McNemar p-value (LR vs baseline): {result.pvalue:.4f}')

# Calculate CI for the difference in proportions with continuity correction
diff = (b - c) / n  # Difference in proportions
z = norm.ppf(0.975)  # 97.5th percentile for 95% CI (two-tailed)
ci_lower = diff - z*np.sqrt((b+c-(b-c)**2/n)/(n**2))
ci_upper = diff + z*np.sqrt((b+c-(b-c)**2/n)/(n**2))

print(f'95% CI for difference in proportions (LR - Baseline): [{ci_lower:.4f}, {ci_upper:.4f}]')

Contingency Table:
[[ 17  12]
 [ 28 111]]
McNemar p-value (LR vs baseline): 0.0166
95% CI for difference in proportions (LR - Baseline): [-0.1676, -0.0229]
